In [31]:
%matplotlib inline
import random
from time import sleep

import gym
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

env = gym.make('Breakout-v0')

In [34]:
done = False
state = env.reset()
# _input = tf.placeholder(shape=[None, 128], dtype=tf.float32)
# conv1d = tf.nn.conv1d([_input], [1,1], stride=32,padding='SAME')
print(state.shape)
env.render()
# with tf.Session as sess:
#     x = tf.run([conv1d], feed_dict({"_input":state}))
#     print(x)

(210, 160, 3)


In [ ]:
class Network:
    def __init__(self):
                
        self.imw, self.imh, self.imc = (210, 160, 3)
        self.input_state = tf.placeholder(tf.float32, shape=[None, self.imw * self.imh * self.imc])
        self.x_image = tf.reshape(self.input_state, [-1, self.imw, self.imh, self.imc])

        self.conv1 = tf.layers.conv2d(self.x_image, 16, 5, activation=tf.nn.relu)
        self.max1 = tf.layers.max_pooling2d(self.conv1, 2, 2)

        self.conv2 = tf.layers.conv2d(self.max1, 8, 5, activation=tf.nn.relu)
        self.max2 = tf.layers.max_pooling2d(self.conv2, 2, 2)

        self.conv3 = tf.layers.conv2d(self.max2, 4, 5, activation=tf.nn.relu)
        self.max3 = tf.layers.max_pooling2d(self.conv3, 2, 2)

        self.flattern = tf.contrib.layers.flatten(self.max3)
        self.fc1 = tf.contrib.layers.fully_connected(self.flattern, 700)
        self.fc2 = tf.contrib.layers.fully_connected(self.fc1, 350)
        self.fc3 = tf.contrib.layers.fully_connected(self.fc2, 120)
        self.fc4 = tf.contrib.layers.fully_connected(self.fc3, 60)
        self.fc5 = tf.contrib.layers.fully_connected(self.fc4, 10)
        self.pred_Q = tf.contrib.layers.fully_connected(self.fc5, 4)
        self.select_action = tf.argmax(self.pred_Q, 1)
        
        self.expect_Q = tf.placeholder(shape=[None, 4], dtype=tf.float32)
        self.loss = tf.losses.mean_squared_error(self.expect_Q, self.pred_Q)
        self.trainer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(self.loss)
        
    def predict(self, sess, state):
        select_action, pred_Q = sess.run([self.select_action, self.pred_Q], \
                                         feed_dict={self.input_state:state})
        return select_action, pred_Q
    
    def update(self, sess, X, Y):
        loss, _ = sess.run([self.loss, self.trainer], feed_dict={self.input_state:X, self.expect_Q:Y})
        return loss
    
class Agent:
    def __init__(self, beta=0.9, discount=0.3, batch_iter=32, batch_size=16, max_memory=1000):
        self.sess = tf.Session()
        self.net = Network()
        self.beta = beta
        self.experience_memory = list()
        self.max_memory = max_memory
        self.batch_size = batch_size
        self.batch_iter = batch_iter
        self.discount = discount
        
    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        self.sess.close()
    
    def initvar(self):
        self.sess.run(tf.global_variables_initializer())
        
    def memory(self, pack):
        if len(self.experience_memory) > self.max_memory:
            self.experience_memory.pop(0)
        self.experience_memory.append( pack )
    
    def predict(self, state):
        is_random = False
        select_action, pred_Q = self.net.predict(self.sess, state)
        if np.random.rand(1) < self.beta:
            is_random = True
            select_action = [env.action_space.sample()]
        return select_action, pred_Q, is_random
    
    
    def replay_experience(self):
        losses = list()
        for batchiter in range(self.batch_iter):
            X = list()
            Y = list()
            batch = random.sample(self.experience_memory, \
                min(len(self.experience_memory), self.batch_size))

            for state, action, reward, next_state, done in batch:
                pq = self.predict([state])[1]
                if done:
                    pq[0, action] = reward
                else:
                    pq[0, action] = reward + self.discount * np.max(self.predict([next_state])[1])
                X.append(state)
                Y.append(pq[0])
            loss = self.net.update(sess, np.array(X), np.array(Y))
            losses.append( loss )
        return np.mean(losses)



In [ ]:
nepisode = 1

with Agent() as agent:
    agent.initvar()
    for episode in range(nepisode+1):
        done = False
        step = 0
        state = env.reset()
        while not done:
            action, pred_Q, _ = agent.predict([state])
            next_state, reward, done, _ = env.step(action[0])
            
            # Big Penalty to Agent
#             if done: 
#                 reward = -100

            agent.memory( (state, action, reward, next_state, done) )
            
            step += 1
            state = next_state
        steps.append(step)
        agent.beta = agent.beta*0.7    
        
        if episode%10 == 0 and episode != 0:
            loss = agent.replay_experience()
            print("in Episode {:4d} mean step: {:.2f}, Agent Replay Experience, get loss: {:.2f} "\
                      .format(episode, np.mean(steps[episode-10:episode+1]), loss))

    
    step = 0
    done = False
    state = env.reset()
    while not done:
        action, Q, _ = agent.predict([state])
        state, reward, done, _ = env.step(action[0])
        step += 1
        env.render()
        sleep(0.1)
    print('Bot play game and archive {} step'.format(step))
    env.close()
    